In [1]:
from pymongo import MongoClient

# Create a connection to the MongoDB server
client = MongoClient('localhost', 27017)

# Connect to a database (it will be created if it doesn't exist)
db = client['sparkplug']

In [2]:
# Connect to a collection (it will be created if it doesn't exist)
collection = db['test']

# Example: Insert a document
collection.insert_one({"name": "John Doe", "age": 30})



InsertOneResult(ObjectId('656a472084168a704cdde54f'), acknowledged=True)

In [3]:
# Example: Retrieve documents
for doc in collection.find():
    print(doc)

{'_id': ObjectId('656a469786a6954445bb9d3f'), 'name': 'John Doe', 'age': 30}
{'_id': ObjectId('656a46f686a6954445bb9d40'), 'name': 'John Doe', 'age': 30}
{'_id': ObjectId('656a472084168a704cdde54f'), 'name': 'John Doe', 'age': 30}
